## 쓰레기통 데이터셋 가공 가이드

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import requests
import re

In [14]:
df = pd.read_excel("서울특별시 가로쓰레기통 현황_202106.xlsx")
df = df.drop(df.columns[[0]], axis=1)
df = df.rename(columns=df.iloc[0])
df = df.drop(df.index[0])

In [2]:
df.head()

,연번,자치구명,도로명,세부 위치(상세 주소),설치 지점,수거 쓰레기 종류(일반 쓰레기 / 재활용 쓰레기)
1,1,종로구,사직로,경복궁역 4번출구,지하철역 입구,일반쓰레기
2,2,종로구,사직로,경복궁역 4번출구,지하철역 입구,재활용
3,3,종로구,자하문로,자하문로 28,도로(가로)변,일반쓰레기
4,4,종로구,자하문로,자하문로 28,도로(가로)변,재활용
5,5,종로구,자하문로,자하문로 44,도로(가로)변,일반쓰레기


### 1차 가공

1. '설치지점'컬럼에 특수기호 제거(①, ②, ③, ④, ⑤, ⑥, ⑦, ⑧, ⑨)

[수거 쓰레기 종류 컬럼]
2. '도봉구' 수거쓰레기 종류 '일반'으로 통일
3. '일반'이 포함된 단어 => 일반
4. '재활용'이 포함된 단어 => 재활용
5. '기타'가 포함된 단어 => 일반
6. 특수기호 제거(⑥, ①, ②)


7. 동일 주소가 중복으로 들어간 row 하나로 합치기

In [15]:
# 도봉구 수거 쓰레기 종류 => "일반"쓰레기로 통일
# ⑥ ① ② 제거
# "일반"이 포함된 단어 => 일반
# "재활용"이 포함된 단어 => 재활용
# "기타"가 포함된 단어 => 일반
need_deleted = ['①', '②', '③', '④', '⑤', '⑥', '⑦', '⑧', '⑨']
gu_list = df['자치구명']
address_list = df['세부 위치(상세 주소)']
setted_list = df['설치 지점']
tr_list = df['수거 쓰레기 종류(일반 쓰레기 / 재활용 쓰레기)']

new_address_list = []
new_tr_list = []
new_setted_list = []
for tr, address, gu, setted in zip(tr_list, address_list, gu_list, setted_list):
    new_tr = tr.replace("⑥", "").replace("①", "").replace("②", "").replace(" ", "").split("+")
    new_setted = str(setted).replace("①", "").replace("②", "").replace("③", "").replace("④", "").replace("⑤", "").replace("⑥", "").replace("⑦", "").replace("⑧", "").replace("⑨", "")
        
    tmp = []
    if gu == "도봉구":
        tmp.append("일반")
    else:
        for tr2 in new_tr:
            if "일반" in tr2: tmp.append("일반")
            elif "재활용" in tr2: tmp.append("재활용")
            elif "기타" in tr2: tmp.append("일반")
            else: tmp.append(tr2)
    new_tr_list.append(tmp)
    new_setted_list.append(new_setted)

    if address == "구풍문여고 앞":
        new_address = address.replace("구풍문여고 앞", "서울공예박물관")
        new_address_list.append(new_address)
    else:
        new_address_list.append(address)

df['수거 쓰레기 종류(일반 쓰레기 / 재활용 쓰레기)'] = new_tr_list
df['세부 위치(상세 주소)'] = new_address_list
df['설치 지점'] = new_setted_list

- 동일주소로 중복된 위치의 쓰레기통 유형 통일해서 DataFrame 새로 만들기

In [16]:
gu_name_list = []
road_name_list = []
address_name_list = []
setted_name_list = []
new_bin_bag_list = []

index_list = df['연번']
gu_list = df['자치구명']
road_list = df['도로명']
address_list = df['세부 위치(상세 주소)']
setted_list = df['설치 지점']
bin_bag_list = df['수거 쓰레기 종류(일반 쓰레기 / 재활용 쓰레기)']
unique_address_dic = {} # 주소 : 인덱스(데이터 위치 rows)
for index, gu, road, address, setted, bins in zip(index_list, gu_list, road_list, address_list, setted_list, bin_bag_list):
    if address not in unique_address_dic.keys(): # 동일 주소가 없는 경우
        gu_name_list.append(gu)
        road_name_list.append(road)
        address_name_list.append(address)
        setted_name_list.append(setted)
        new_bin_bag_list.append(bins)
        unique_address_dic[address] = len(new_bin_bag_list) - 1
    else: # 동일 주소가 이미 있는 경우
        find_index = unique_address_dic[address] # 동일 주소가 등록된 인덱스 위치
        new_bin_bag_list[find_index].extend(bins)

new_df = pd.DataFrame({
                        'region': gu_name_list,
                        'roads': road_name_list,
                        'details': address_name_list,
                        'points': setted_name_list,
                        'type': new_bin_bag_list
                       })

In [5]:
new_df

,region,roads,details,points,type
0,종로구,사직로,경복궁역 4번출구,지하철역 입구,"[일반, 재활용]"
1,종로구,자하문로,자하문로 28,도로(가로)변,"[일반, 재활용]"
2,종로구,자하문로,자하문로 44,도로(가로)변,"[일반, 재활용]"
3,종로구,자하문로,자하문로 68(효자동 정류소),"정류장(버스, 택시 등)","[일반, 재활용]"
4,종로구,효자로,청와대 분수대(사랑채),광장 등 다중집합장소,"[일반, 재활용]"
...,...,...,...,...,...
3683,도봉구,노해로,창동 135-26,창동지하차도 이마트 건너편 횡단보도,[일반]
3684,도봉구,해등로,해등로3길 41,삼환빌라 앞 횡단보도,[일반]
3685,도봉구,해등로,해등로 103,창원초등학교 앞 횡단보도,[일반]
3686,도봉구,도봉로,도봉동 620-23,신도봉사거리 도봉중학교방향 횡단보도,[일반]


### 카카오 API 호출 시 검색용 주소 
- 도로명 주소에서 '00로'만 추출한 뒤, '00로'가 상세주소 내용에 있으면 상세주소로만 호출
- 만약 '00로'가 상세주소안에 없다면 '00로' + 상세주소로 호출(roads가 NaN인 경우가 있으니 이때는 상세주소로 호출하도록 예외처리)

### 카카오 API를 통해 추가할 새로운 컬럼
- 전체 '도로명주소'
- x좌표 : y
- y좌표 : x
- 주소가 잘 검색되는 row의 roads에 도로명 변경(검색결과와 기존 값이 같지 않은 경우)

In [20]:
index_list = [i for i in range(len(new_df['region']))]
region_list = new_df['region']
road_list = new_df['roads']
address_list = new_df['details']

none_data_list = [] # 검색값을 받아오지 못한 row의 인덱스
new_road_list = [] # API에서 받은 도로명 주소 -> 따로 컬럼 만들기(일단 만들고 나중에 없애기)
full_address_list = [] # API를 통해 검색된 풀 주소(정확한 주소)
x_list, y_list = [], []
for idx, region, road, address in tqdm(zip(index_list, region_list, road_list, address_list)):
    try:
        road = re.sub("[0-9 ]*길", "", road)
        road = road.replace(" ", "")
        if road in address:
            query = f"{address}"
            res = requests.get("https://dapi.kakao.com//v2/local/search/address.json", headers={'Authorization': 'KakaoAK de826e450389e974c1075234987b2af7'}, params={"query": query})
        elif road not in address and road != None:
            query = f"{road} {address}"
            res = requests.get("https://dapi.kakao.com//v2/local/search/address.json", headers={'Authorization': 'KakaoAK de826e450389e974c1075234987b2af7'}, params={"query": query})
        response = res.json()

        if response['meta']['total_count'] > 0: # 검색 결과가 잘 나온 경우
            x = response['documents'][0]['y']
            y = response['documents'][0]['x']
            full_address = response['documents'][0]['road_address']['address_name']
            if full_address == None:
                full_address = response['documents'][0]['address_name']
            road_name = response['documents'][0]['road_address']['road_name']


        else: # 검색했는데 값이 없는 경우
            none_data_list.append(idx)
            x, y, full_address, road_name = None, None, None, None
        
    except:
        none_data_list.append(idx)
        x, y, full_address, road_name = None, None, None, None

    x_list.append(x)
    y_list.append(y)
    new_road_list.append(road_name)
    full_address_list.append(full_address)

new_df['full_address'] = full_address_list
new_df['correct_roads'] = new_road_list
new_df['x'] = x_list
new_df['y'] = y_list

3688it [48:29,  1.27it/s]


In [21]:
print("API로 검색되지 않는 주소를 가진 rows의 개수 >>", len(none_data_list))

API로 검색되지 않는 주소를 가진 rows의 개수 >> 2185


In [32]:
new_df.sample(10)

,region,roads,details,points,type,full_address,correct_roads,x,y
860,성북구,성북로,서울다원학교 버스정류장 (08-223),"정류소(버스, 택시 등)",[일반],None,None,None,None
2653,서초구,NaN,고속터미널교차로 G14번 출구 앞 반포쇼핑타운8동 앞,nan,"[재활용, 재활용]",None,None,None,None
204,중구,퇴계로,성동고등학교 앞 버스정류장 02-180,"정류소(버스, 택시 등)","[일반, 재활용]",None,None,None,None
3541,도봉구,노해로,노해로69길 15,벽제갈비촌 앞,[일반],서울 도봉구 노해로69길 15,노해로69길,37.6529808344235,127.050643136271
3540,도봉구,노해로,노해로69길 25,강북민물장어 앞,[일반],서울 도봉구 노해로69길 25,노해로69길,37.6535785379122,127.050452367786
3614,도봉구,해등로,해등로 305,우성2차아파트 201동 앞 4거리,"[일반, 일반]",서울 도봉구 해등로 305,해등로,37.6572641441672,127.022686737213
2206,구로구,경인로,경인로319,도로변(횡단보도 포함),[재활용],서울 구로구 경인로 319,경인로,37.4975993752858,126.855601090574
399,용산구,한강대로,"숙대입구역 3,4번출구앞(군선도가게앞)",지하철역 입구,[일반],None,None,None,None
413,용산구,서빙고로,용산세무서(용산공고방면)버스정류장((03262),"정류장(버스,택시 등)",[일반],None,None,None,None
1202,동작구,동작대로,19(한국전력 앞),지하철 옆,"[재활용, 일반]",서울 동작구 동작대로 19,동작대로,37.4783088342982,126.980958500451


In [51]:
new_df[new_df['region']=='강서구']

,region,roads,details,points,type,full_address,correct_roads,x,y
1948,강서구,가로공원로,나누리병원 16-199,버스정류장,[일반],None,None,None,None
1949,강서구,가로공원로,나누리병원 16-200,버스정류장,[일반],None,None,None,None
1950,강서구,강서로,까치산역 16-192,버스정류장,"[일반, 재활용]",None,None,None,None
1951,강서구,강서로,화곡역.현대주유소 16-205,버스정류장,[일반],None,None,None,None
1952,강서구,강서로,화곡중고등학교16-211,버스정류장,[일반],None,None,None,None
...,...,...,...,...,...,...,...,...,...
2172,강서구,양천로,염창동새마을금고 버스정류장(16-247),버스정류장,[일반],None,None,None,None
2173,강서구,허준로,가양초등학교 버스정류장(16-240),버스정류장,[일반],None,None,None,None
2174,강서구,허준로,세현고등학교 버스정류장(16-244),버스정류장,[일반],None,None,None,None
2175,강서구,공항대로,발산역5번출구 버스정류장(16-112),버스정류장,[일반],None,None,None,None


### 2차 가공
- 데이터가 있는 것, 없는 것 분리(	region	roads	details	points	type	full_address	correct_roads	x	y)
- 데이터 없는 DataFrame -> for문 돌면서 검색이 되는 주소로 수동검수하기
- 수동검수가 끝난 데이터 -> 다시 카카오API 요청
- 그러고도 없는 데이터들은 버리기
- 최종적으로 DataFrame을 하나로 합쳐 저장



In [43]:
if np.isnan(df.loc[0]['x']):
    print("Nan")

Nan


In [ ]:
for i in range(2801, 3688):
    if df.loc[i]['full_address'] == None and np.isnan(df.loc[i]['x']) and np.isnan(df.loc[i]['y']):
        print(f"{i}번쨰, 검색결과가 나오지 않는 상세주소 >>", df.at[i, 'roads'], "+", df.at[i, 'details'])
        new_details = input()

        if new_details != "":압구정로 201
            df.at[i, 'details'] = new_details

- 수정한 상세주소 기반 좌표 재요청

In [20]:
query="적선동 88-1(경복궁역 4번출구)"
res = requests.get("https://dapi.kakao.com//v2/local/search/address.json", headers={'Authorization': 'KakaoAK de826e450389e974c1075234987b2af7'}, params={"query": query})
res.json()

{'documents': [{'address': {'address_name': '서울 종로구 적선동 88-1',
    'b_code': '1111010700',
    'h_code': '1111053000',
    'main_address_no': '88',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_h_name': '사직동',
    'region_3depth_name': '적선동',
    'sub_address_no': '1',
    'x': '126.97334713161',
    'y': '37.5760320156834'},
   'address_name': '서울 종로구 적선동 88-1',
   'address_type': 'REGION_ADDR',
   'road_address': None,
   'x': '126.97334713161',
   'y': '37.5760320156834'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [2]:
df = pd.read_json('bins_data2801.json')

In [3]:
for i in tqdm(range(3688)):
    if df.loc[i]['full_address'] == None and np.isnan(df.loc[i]['x']) and np.isnan(df.loc[i]['y']):
        try:
            query = df.loc[i]['details']
            res = requests.get("https://dapi.kakao.com//v2/local/search/address.json", headers={'Authorization': 'KakaoAK de826e450389e974c1075234987b2af7'}, params={"query": query})
            response = res.json()
            if response['meta']['total_count'] > 0: # 상세주소 검색 결과가 잘 나온 경우
                x = response['documents'][0]['y']
                y = response['documents'][0]['x']
                full_address = response['documents'][0]['address_name']
                road_name = response['documents'][0]['road_address']['road_name'] if response['documents'][0]['road_address'] != None else None
            else: # 상세주소로 검색했는데 값이 없는 경우
                x, y = None, None
                full_address, road_name = None, None
                
        except:
            x, y, full_address, road_name = None, None, None, None

        if full_address != None and x != None and y != None:
            df.at[i, 'full_address'] = full_address
            df.at[i, 'x'] = x
            df.at[i, 'y'] = y

100%|██████████| 3688/3688 [22:42<00:00,  2.71it/s]


In [13]:
df.drop(['correct_roads'], axis=1, inplace=True)

In [93]:
df.to_json('bins_data_re2.json', orient='records')

- type : 중복으로 들어간 값 제거
- [일반, 재활용, 일반, 재활용] => [일반, 재활용]

In [10]:
for i in range(3688):
    type_list = df.loc[i, 'type']
    set_type_list = list(set(type_list))
    df.at[i, 'type'] = set_type_list

### 3차 가공
- 'roads'컬럼에 correct_roads값('00로'만 있는 값)으로 대체
- 이후 correct_roads 컬럼은 제거
- 'roads'컬럼이 None인 셀에 도로명 주소 넣기('강남대로' roads 컬럼에 추가)


In [98]:
type(df.loc[i, 'roads'])

str

In [110]:
for i in tqdm(range(3688)):
    try:
        if df.loc[i, 'roads'] == "None":
            for address in str(df.loc[i, 'full_address']).split():
                if address.endswith("로"):
                    roads = address
                    df.at[i, 'roads'] = roads
            
            if df.loc[i, 'roads'] == "None": # '00로'라는 도로명이 없는 경우
                for address in str(df.loc[i, 'full_address']).split():
                    if address.endswith("동"):
                        roads = address
                        df.at[i, 'roads'] = roads

            if df.loc[i, 'roads'] == "None": # '00로'나 '00동'이 없는 경우
                roads = str(df.loc[i, 'full_address']).split()[2]
                roads = re.sub("[0-9가길 ]+", "", roads)
                df.at[i, 'roads'] = roads
    except:
        pass

100%|██████████| 3688/3688 [00:00<00:00, 67355.49it/s]


In [134]:
df[df['roads'] == '기타']

,region,roads,details,points,type,full_address,x,y
662,광진구,기타,정보도서관 앞 낭만의 거리 데크,광장 등 다중집합장소,"[재활용, 일반]",None,NaN,NaN
663,광진구,기타,광진교 북단 전망대 앞,광장 등 다중집합장소,[일반],None,NaN,NaN
664,광진구,기타,중랑천 매점 앞(굴다리 입구),광장 등 다중집합장소,"[재활용, 일반]",None,NaN,NaN
665,광진구,기타,중랑천 농구코트 앞,광장 등 다중집합장소,[일반],None,NaN,NaN


In [112]:
df.loc[2654, 'roads']

'잠원로'

In [94]:
df = pd.read_json('bins_data_re2.json')

In [95]:
df.head()

,region,roads,details,points,type,full_address,x,y
0,종로구,사직로,적선동 88-1(경복궁역 4번출구),지하철역 입구,"[재활용, 일반]",서울 종로구 적선동 88-1,126.973347,37.576032
1,종로구,자하문로,자하문로 28,도로(가로)변,"[재활용, 일반]",서울 종로구 자하문로 28,37.578567,126.971970
2,종로구,자하문로,자하문로 44,도로(가로)변,"[재활용, 일반]",서울 종로구 자하문로 44,37.579972,126.971550
3,종로구,자하문로,자하문로 68(효자동 정류소),"정류장(버스, 택시 등)","[재활용, 일반]",서울 종로구 자하문로 68,37.582101,126.970932
4,종로구,효자로,효자로13길 45,광장 등 다중집합장소,"[재활용, 일반]",서울 종로구 효자로13길 45,126.973059,37.582545


- region: 끝자리 단어가 '구'가 아니면 '구' 붙이기
- roads: 끝자리 단어가 '로'가 아니면 '로' 붙이기

In [121]:
df.loc[2560, 'roads']

'대림동'

In [ ]:
for i in range(3688):
    if not df.loc[i, 'roads'].endswith("로") and not df.loc[i, 'roads'].endswith("동") and df.loc[i, 'roads'] != "None":
        roads = df.loc[i, 'roads']
        if roads == "이화여대":
            roads = "이화여대길"
        elif "밤고개" in roads:
            roads = "헌릉로"
        roads = roads.replace(" ", "").replace("-", "").replace("건너편", "")
        df.at[i, 'roads'] = roads
        print("도로명 >> ", df.loc[i, 'roads'])

In [146]:
df.sample(15)

,region,roads,details,points,type,full_address,x,y
57,종로구,새문안로,서울역사박물관 버스정류장(01-008),도로(가로)변,[일반],None,NaN,NaN
3522,도봉구,마들로,마들로 671,도봉중학교 앞,[일반],서울 도봉구 마들로 671,37.670884,127.045872
3542,도봉구,노해로,노해로 383,국민은행 앞,[일반],서울 도봉구 노해로 383,37.652584,127.050731
156,중구,을지로,"을지로입구, 로얄호텔 버스정류장 02-156","정류소(버스, 택시 등)","[재활용, 일반]",None,NaN,NaN
3058,강남구,삼성로,대치동 988-14,도로변(횡단보도 포함),"[재활용, 일반]",서울 강남구 대치동 988-14,127.061127,37.499390
1666,서대문구,충정로,충정로 충정로역 (13046),정류장(버스 택시 등),[일반],None,NaN,NaN
2933,강남구,삼성로,노브랜드 강남삼성점,도로변(횡단보도 포함),"[재활용, 일반]",None,NaN,NaN
1797,마포구,어울마당로,어울마당로 65 KT 앞,도로(가로)변,[일반],서울 마포구 어울마당로 65,37.550958,126.921063
3213,송파구,올림픽로,올림픽회관(24-231),"정류소(버스, 택시 등)",[일반],None,NaN,NaN
2802,강남구,봉은사로,쿨마트 논현점\n(아크로힐스 논현아파트 104동 앞),"정류소(버스, 택시 등)","[재활용, 일반]",None,NaN,NaN


### 마지막 데이터 정리
- 'roads', 'full_address', 'x',	'y' 중 하나라도 값이 None이면 해당 row 삭제하기

In [148]:
np.isnan(df.loc[57, 'x'])

True

In [150]:
drop_list = []
for i in tqdm(range(3688)):
    if df.loc[i, 'roads'] == "None" or df.loc[i, 'full_address'] == "None" or np.isnan(df.loc[i, 'x']) or np.isnan(df.loc[i, 'y']):
        drop_list.append(i)

final_df = df.drop(drop_list)

100%|██████████| 3688/3688 [00:00<00:00, 20695.40it/s]


- 위도, 경도 바뀐 row 스위칭

In [184]:
final_df = final_df.reset_index(drop=True) # 인덱스 초기화

In [186]:
for i in range(2292):
    x = final_df.loc[i, 'x']
    y = final_df.loc[i, 'y']
    if y > x:
        final_df.at[i, 'x'] = y
        final_df.at[i, 'y'] = x

In [200]:
final_df.sample(15)

,region,roads,details,points,type,full_address,x,y
720,동작구,알마타,알마타길 6,가로변,"[재활용, 일반]",서울 동작구 알마타길 6,126.924954,37.509857
1295,양천구,중앙로,해성산부인과 버스정류장 앞,"정류소(버스, 택시 등)","[재활용, 일반]",서울 양천구 중앙로 228,126.854602,37.516991
102,중구,퇴계로,충무로1가 24-1(명동역 6번출구),지하철역 입구,"[재활용, 일반]",서울 중구 충무로1가 24-1,126.984520,37.561053
560,강북구,도봉로,도봉로 352 앞 (3개 1조),지하철역 입구,[일반],서울 강북구 도봉로 352,127.027033,37.638675
1620,구로구,서해안로,항동 113-5,도로변(횡단보도 포함),[재활용],서울 구로구 항동 113-5,126.822112,37.479831
1408,강서구,금낭화로,금낭화로 135 방화역 3번출구 택시정류장,지하철입구,[일반],서울 강서구 금낭화로 135,126.812438,37.577445
678,동작구,매봉로,165번지(수학원),가로변,[재활용],서울 동작구 매봉로 165,126.951614,37.511856
2236,도봉구,노해로,노해로 224,쌍문3동 어린이집 앞,[일반],서울 도봉구 노해로 224,127.032948,37.650918
829,은평구,갈현로,갈현로152 스마트부동산앞(신축),도로변(횡단보도 포함),[일반],서울 은평구 갈현로 152,126.911425,37.611396
1630,금천구,시흥대로,63길48,"정류소(버스, 택시 등)",[일반],서울 금천구 시흥대로63길 48,126.897069,37.455440


In [203]:
final_df.to_json('bins_data_final.json', orient = 'records', indent = 4, force_ascii = False)

In [202]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2292 entries, 0 to 2291
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   region        2292 non-null   object 
 1   roads         2292 non-null   object 
 2   details       2292 non-null   object 
 3   points        2292 non-null   object 
 4   type          2292 non-null   object 
 5   full_address  2292 non-null   object 
 6   x             2292 non-null   float64
 7   y             2292 non-null   float64
dtypes: float64(2), object(6)
memory usage: 143.4+ KB
